# Data Drift Detection in Airflow — Technical Documentation

---

## 1. **Purpose & Importance**

**Data drift detection** is a critical part of a production ML pipeline. It monitors whether the statistical properties of incoming data have changed compared to the training data. If drift is detected, it can trigger retraining, alert stakeholders, or block predictions to maintain model reliability.

- **Why:**  
  - Prevents model degradation due to changing data distributions.
  - Ensures predictions remain accurate and trustworthy.
  - Automates monitoring for compliance and auditability.

---

## 2. **Implementation in Airflow**

### **Typical DAG Structure**

- **DAG File:** `dags/data_drift_check.py`
- **Operators:** Usually a `PythonOperator` running a drift detection function.
- **Schedule:** Can be run daily, hourly, or after new data ingestion.

### **Core Steps**

1. **Load Reference Data**
   - Load training data statistics or a sample of the original training set.

2. **Load New Data**
   - Load the latest batch of data for scoring.

3. **Run Drift Detection**
   - Apply statistical tests (e.g., Kolmogorov-Smirnov, Population Stability Index, Chi-square) to compare distributions.

4. **Log Results**
   - Write drift status to a file (`drift_detected.txt`, `drift_log.txt`, or set a flag).
   - Optionally trigger downstream tasks (e.g., retraining DAG).

5. **Alert/Trigger**
   - If drift is detected, send alerts or trigger retraining.

---

## 3. **Example Airflow DAG Code**



In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import pandas as pd
from scipy.stats import ks_2samp

def check_data_drift():
    # Load reference (training) and new data
    ref_df = pd.read_csv('/data/preprocessed_data.csv')
    new_df = pd.read_csv('/data/new_batch.csv')

    drift_report = {}
    drift_detected = False

    # Example: Check drift for each numerical column
    for col in ref_df.select_dtypes(include='number').columns:
        stat, p_value = ks_2samp(ref_df[col].dropna(), new_df[col].dropna())
        drift_report[col] = {'ks_stat': stat, 'p_value': p_value}
        if p_value < 0.05:  # Significant drift
            drift_detected = True

    # Log drift status
    with open('/data/drift_detected.txt', 'w') as f:
        f.write('drift_detected' if drift_detected else 'no_drift')

    # Optionally, log detailed report
    pd.DataFrame(drift_report).T.to_csv('/data/drift_log.txt')

    # Optionally, trigger retraining or alert

with DAG('data_drift_detection', start_date=datetime(2023, 1, 1), schedule_interval='@daily') as dag:
    drift_task = PythonOperator(
        task_id='check_data_drift',
        python_callable=check_data_drift
    )



---

## 4. **Technical Notes**

- **Statistical Tests:**  
  - *KS Test:* For continuous features.
  - *PSI (Population Stability Index):* For categorical or binned features.
  - *Chi-square:* For categorical features.

- **Thresholds:**  
  - Commonly, p-value < 0.05 indicates significant drift.

- **Artifacts:**  
  - Drift status and logs are saved for audit and downstream automation.

- **Extensibility:**  
  - Add email/SMS alerts.
  - Trigger retraining DAG if drift is detected.
  - Integrate with monitoring dashboards.

---

## 5. **Best Practices**

- Monitor both input features and target variable.
- Log all drift checks for compliance.
- Automate retraining only after human review if possible.
- Version all drift reports and status flags.

---

## 6. **Summary Table**

| Step                | Purpose                       | Implementation/Reason           |
|---------------------|------------------------------|---------------------------------|
| Load Reference Data | Baseline for comparison      | Ensures valid drift detection   |
| Load New Data       | Data to monitor              | Detects changes in distribution |
| Run Drift Tests     | Statistical comparison       | Identifies significant changes  |
| Log Results         | Audit and automation         | Enables traceability            |
| Trigger Actions     | Retraining/alerting          | Maintains model reliability     |

---

**In summary:**  
Data drift detection in Airflow automates the monitoring of data changes, ensuring your ML models remain accurate and reliable in production. The process is modular, auditable, and can be extended for alerts and retraining.